In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [6]:
df = pd.read_csv('streamer_df')

In [148]:
def get_data(url, pages):
    import time
    t=5
    df = parse_html(f'{url}=1') # getting the first page
    pages = range(2,pages)
    for page in pages: # getting all other pages, defined by argument
        print(page)
        new_df = parse_html(f"{url}{page}")
        df = df.append(new_df, ignore_index=True)
        time.sleep(t)
    return df

def parse_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    df=pd.read_html(str(soup), flavor='bs4')[0]
    return df

# creates a dataframe for social media plattforms scraped from twitch for a given df with usernames in it
def get_social(df):
    ''' this function scrapes the URL to youtube and twitter for a username-list'''
    import time
    t=5
    usernames = df.username.to_list()
    options = webdriver.ChromeOptions()
    options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
    chrome_driver_binary = "/Users/frckles/drivers/chromedriver"
    dict = {}
    for user in usernames:
        URL= f'https://www.twitch.tv/{user}/about'
        driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
        driver.get(URL)
        soup=BeautifulSoup(driver.page_source)
        twitter = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "twitter.com/" in a["href"]]))
        youtube = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "youtube.com/" in a["href"]]))
        instagram = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "instagram" in a["href"]]))
        discord = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "discord" in a["href"]]))
        if twitter:
            twitter = twitter[0]
        if youtube:
            youtube = youtube[0]
        if instagram:
            instagram = instagram[0]
        if discord:
            discord = discord[0]
        dict[user] = [twitter, youtube, instagram, discord]
        time.sleep(t)
    df = pd.DataFrame.from_dict(dict, orient='index')
    df['Twitter'] = df[0].apply(lambda x: None if x == [] else x)
    df['YouTube'] = df[1].apply(lambda x: None if x == [] else x)
    df['Instagram'] = df[2].apply(lambda x: None if x == [] else x)
    df['Discord'] = df[3].apply(lambda x: None if x == [] else x)
    df.index.name = 'username'
    df = df.reset_index()
    return df[['username', 'Twitter', 'YouTube', 'Instagram', 'Discord']]

# transfer stuff to csv
# df.to_csv(r'/Users/home/code/Awesimous/StarFish/notebooks/put_name', index=False)

In [149]:
socials = get_social(df.sample(3))

<ipython-input-148-1de47f7e76cf>:32: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)


In [150]:
df_socs = socials.copy()
df_socs

,username,Twitter,YouTube,Instagram,Discord
0,RocketBaguette,https://twitter.com/rocketbaguette,http://youtube.com/rocketbaguette,https://instagram.com/rocketbaguette,http://discord.gg/rocketbaguette
1,Japczan,None,https://www.youtube.com/japczan,https://www.instagram.com/japczanek/,https://discord.gg/zcud4vqqck
2,VaporaDark,https://twitter.com/vaporadark,https://www.youtube.com/user/vaporadark?sub_co...,None,https://discord.gg/vaporadark


In [152]:
def get_social_id(link):
    '''Extract streamer identifier at the end of their social media links'''
    if link[-1] == '/':
        link = link[:-1]
    start = link.rfind('/')
    id = link[start + 1:]

    return id


def parse_streamer_socials(socials_df):
    '''Return a dataframe with relevant ids per treamer and platform'''
    for (column_name, column_data) in socials_df.iteritems():
        if column_name != 'Discord':
            socials_df[column_name] = column_data.apply(lambda x: get_social_id(x) if x else None)
            if column_name == 'YouTube':
                socials_df[column_name] = socials_df[column_name].apply(lambda x: x.split('?')[0])
            
        
    return socials_df

In [153]:
l = df_soc.iloc[0,3]
get_streamer_identifier(l)

'lily_off_valley'

In [154]:
parse_streamer_socials(df_socs)
df_socs

,username,Twitter,YouTube,Instagram,Discord
0,RocketBaguette,rocketbaguette,rocketbaguette,rocketbaguette,http://discord.gg/rocketbaguette
1,Japczan,None,japczan,japczanek,https://discord.gg/zcud4vqqck
2,VaporaDark,vaporadark,vaporadark,None,https://discord.gg/vaporadark
